Esoph Dataset, see https://galeascience.wordpress.com/2016/05/05/linear-models-for-predicting-esophagus-cancer-likelihood/

Step 1: get it into torch tensor

In [6]:
import statsmodels.api as sm
import pandas

esoph = sm.datasets.get_rdataset('esoph')


In [14]:
df = esoph.data
gdef_1 = df.columns[0]
gdef_2 = df.columns[1]
gdef_3 = df.columns[2]

gdef_1_vals = df[gdef_1].unique()
gdef_2_vals = df[gdef_2].unique()
gdef_3_vals = df[gdef_3].unique()

gdef_1_dict = { gdef_1_vals[i] : i for i in range(0, len(gdef_1_vals) ) }
gdef_2_dict = { gdef_2_vals[i] : i for i in range(0, len(gdef_2_vals) ) }
gdef_3_dict = { gdef_3_vals[i] : i for i in range(0, len(gdef_3_vals) ) }

In [15]:
cleaned_df = df.replace({ gdef_1 : gdef_1_dict, gdef_2 : gdef_2_dict, gdef_3 : gdef_3_dict})


,agegp,alcgp,tobgp,ncases,ncontrols
0,0,0,0,0,40
1,0,0,1,0,10
2,0,0,2,0,6
3,0,0,3,0,5
4,0,1,0,0,27


In [17]:
import torch

t = torch.from_numpy(cleaned_df.to_numpy())


In [23]:
groups = t.size()[0]
print(groups)

88


In [24]:
cases = torch.sum(t[:,3]).item()
print(cases)

200


In [25]:
controls = torch.sum(t[:,4]).item()
print(controls)

975


In [35]:
highest_alc_gr = len(gdef_2_vals) - 1
lowest_alc_gr = 0

acl_gr = t[:,1]
highest_acl_rows = torch.index_select(t, 0, torch.where(acl_gr == highest_alc_gr, torch.ones(1), torch.zeros(1)).nonzero()[:,0])
lowest_acl_rows = torch.index_select(t, 0, torch.where(acl_gr == lowest_alc_gr, torch.ones(1), torch.zeros(1)).nonzero()[:,0])

highest_alc_cancer_count = torch.sum(highest_acl_rows[:,3]).item()
highest_alc_control_count = torch.sum(highest_acl_rows[:,4]).item()
lowest_alc_cancer_count = torch.sum(lowest_acl_rows[:,3]).item()
lowest_alc_control_count = torch.sum(lowest_acl_rows[:,4]).item()

highest_alc_cancer_prob = highest_alc_cancer_count / (highest_alc_cancer_count + highest_alc_control_count)
lowest_alc_cancer_prob = lowest_alc_cancer_count / (lowest_alc_cancer_count + lowest_alc_control_count)

print(highest_alc_cancer_prob)
print(lowest_alc_cancer_prob)

0.4017857142857143
0.06531531531531531


In [42]:
at_least_10g_tob = 1
case_col = t[:,3]
control_col = t[:,4]

has_case = case_col > 0
has_control = control_col > 0
has_at_least10g = t[:,2] > 0

all_cases_rows_10g_tob_least = torch.index_select(t, 0, torch.where(has_case & has_at_least10g, torch.ones(1), torch.zeros(1)).nonzero()[:,0]) 
all_control_rows_10g_tob_least = torch.index_select(t, 0, torch.where(has_control & has_at_least10g, torch.ones(1), torch.zeros(1)).nonzero()[:,0]) 

prob_10g_cases = torch.sum(all_cases_rows_10g_tob_least[:,3]).item() / cases
prob_10g_control = torch.sum(all_control_rows_10g_tob_least[:,4]).item() /controls

print(prob_10g_cases)
print(prob_10g_control)

0.61
0.46153846153846156


In [43]:
print(highest_alc_cancer_count / cases)

0.225


In [44]:
highest_tob_gr = len(gdef_3_vals) - 1

tob_gr = t[:,2]
highest_tob_rows = torch.index_select(t, 0, torch.where(tob_gr == highest_tob_gr, torch.ones(1), torch.zeros(1)).nonzero()[:,0])

highest_tob_cancer_count = torch.sum(highest_tob_rows[:,3]).item()
print(highest_tob_cancer_count / cases)

0.155


In [48]:
high_tob = tob_gr == highest_tob_gr
high_alc = acl_gr == highest_alc_gr

highest_all_rows = torch.index_select(t, 0, torch.where(high_tob & high_alc, torch.ones(1), torch.zeros(1)).nonzero()[:,0])

highest_all_cancer_count = torch.sum(highest_all_rows[:,3]).item()
print(highest_all_cancer_count / cases)

0.05


In [49]:
highest_or_rows = torch.index_select(t, 0, torch.where(high_tob | high_alc, torch.ones(1), torch.zeros(1)).nonzero()[:,0])

highest_or_cancer_count = torch.sum(highest_or_rows[:,3]).item()
print(highest_or_cancer_count / cases)

0.33


In [50]:
print(highest_alc_control_count / controls)

0.06871794871794872


In [53]:
print((highest_alc_cancer_count / cases) / (highest_alc_control_count / controls))

3.2742537313432836


In [54]:
highest_tob_control_count = torch.sum(highest_tob_rows[:,4]).item()
print(highest_tob_control_count / controls)

0.0841025641025641


In [55]:
highest_all_control_count = torch.sum(highest_all_rows[:,4]).item()
print(highest_all_control_count / controls)

0.013333333333333334


In [57]:
highest_or_control_count = torch.sum(highest_or_rows[:,4]).item()
print(highest_or_control_count / controls)

0.13948717948717948


In [58]:
print((highest_or_cancer_count / cases)/(highest_or_control_count / controls))

2.365808823529412
